In [1]:
%load_ext autoreload
%autoreload 2

Презентация показывает основные концепции, используемые в библиотеке в ознакомительном формате без тонкостей реализации и т.д.

## Sampler (идея, цель, примеры использования)

### Идея:
- обертка над всеми данными, хранит в себе train, oos, oot (и признаки, таргет)
- бесконечный источник данных из того же распределения, что и данные пришедшие на валидацию. Реализовываться это может подкапотом как угодно (скорее всего KFold, Bootstraping или Resampling)

### Цель:

у некоторых моделей есть ограничения на *resampling* данных (разбивать на train и oos не строки, а людей, определяя их по ИНН). Данный класс позволяет абстрагироваться от этих тонкостей, предоставляя единый интерфейс.


### Типичные примеры исполльзования

In [2]:
# Создаем данные
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from IPython.display import display


X, y = make_classification(n_samples=1000, n_classes=2,
                           n_features=3, n_informative=2,
                           n_redundant=0, random_state=0)
X = pd.DataFrame(X)
y = pd.Series(y)
X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

print('Исходные данные: ')
display(X_train[:3])
display(X_test[:3])

Исходные данные: 


,0,1,2
290,-0.844349,0.504995,0.825358
662,-1.144246,1.103504,-1.392602
864,-0.662642,-0.256870,1.054222


,0,1,2
715,1.460184,0.789905,2.341212
823,0.239886,1.483129,-0.447120
404,0.846301,-0.152794,1.686599


In [3]:
# Создаем sampler
from sbe_vallib.sampler.supervised_sampler import SupervisedSampler

sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train},
                            oos={'X': X_test, 'y_true': y_test},
                            oot=None)

получаем данные по атрибуту ```train, oos, oot``` (под капотом там лежат ```property```)

In [4]:
display(sampler.train['X'][:4])
display(sampler.oos['X'][:4])

,0,1,2
290,-0.844349,0.504995,0.825358
662,-1.144246,1.103504,-1.392602
864,-0.662642,-0.256870,1.054222
978,-1.503873,0.655924,0.533744


,0,1,2
715,1.460184,0.789905,2.341212
823,0.239886,1.483129,-0.447120
404,0.846301,-0.152794,1.686599
439,-1.533187,1.320794,0.647763


In [5]:
# Фиксируем seed и способ генерации новой выборки
sampler.set_state(seed=5, gen_method='bootstrap')

#теперь по аттрибутам train, oos, oot у нас новые выборки, полученные бутстрапированием исходных
display(sampler.train['X'][:4])
display(sampler.oos['X'][:4])


,0,1,2
5,0.486414,1.497715,-0.184009
213,1.203531,-1.320525,1.571972
945,1.948291,-1.093895,-2.044562
330,-0.789695,0.374155,-1.335048


,0,1,2
576,1.942820,0.139988,1.783529
362,-0.415896,0.514004,1.741516
238,1.220240,-1.505911,0.531213
373,0.722205,-1.044816,-0.348215


In [6]:
# Перезагружаем состояние sampler, чтобы получать снова исходные выборки
sampler.reset()

display(sampler.train['X'][:4])
display(sampler.oos['X'][:4])

,0,1,2
290,-0.844349,0.504995,0.825358
662,-1.144246,1.103504,-1.392602
864,-0.662642,-0.256870,1.054222
978,-1.503873,0.655924,0.533744


,0,1,2
715,1.460184,0.789905,2.341212
823,0.239886,1.483129,-0.447120
404,0.846301,-0.152794,1.686599
439,-1.533187,1.320794,0.647763


## Scorer (идея, цель, примеры использования)

### Идея:

Идея этого класса - посчитать скопом заранее заданные метрики при вызове метода ```scorer.calc_metrics()```

### Цель:

Причина появления: модели могут оцениваются самыми разными способами, все не уложить в тип ```func(y_true, y_pred)```.Например *YOLO* или же в табличных данных ключевая метрика это сумма на ```train``` и ```oos```. С помощью этого класса абстрагируемся от этого в через подачу в скорер всего ```scorer.calc_metrics(model, sampler, **something_else)```, оставляя реализацию в нестандартных случаях на валидатора. 


In [7]:
# создаем scorer
from sbe_vallib.scorer.table_scorer import BinaryScorer
from sbe_vallib.utils.metrics import BINARY_METRICS

scorer = BinaryScorer(metrics=BINARY_METRICS, cutoff=0.1)

In [8]:
# есть метрики, которые работают с классом, а есть которые работают с вероятностями
# scorer внутри себя регулирует, что куда подать
list(BINARY_METRICS.keys())

['accuracy_score', 'precision_score', 'recall_score', 'f1_score', 'gini']

### Типичные примеры исполльзования

In [9]:
# для начала создадим и обучим модель

from sklearn.linear_model import LogisticRegression

sampler.reset()
model = LogisticRegression().fit(sampler.train['X'], sampler.train['y_true'])

In [10]:
scorer.calc_metrics(model=model, sampler=sampler, data_type='oos')

{'accuracy_score': 0.8766666666666667,
 'precision_score': 0.8121546961325967,
 'recall_score': 0.98,
 'f1_score': 0.8882175226586102,
 'gini': 0.9587555555555556}

## Test (идея, цель, примеры использования)

### Идея:

Функция, которая реализует подсчет теста и его оформление. На эту функцию накладываются ограничения на сигнатуру: 

- функция как минимум должна принимать параметры ```model, scorer, sampler, precomputed: dict=None, **kwargs``` 
- функция должна выдать результат работы в следующем формате (привет SberDS)
    ```
    {
        "semaphore": str one of {"gray", "green", "yellow", "red"},
        "result_dict": python object,
        "result_dataframes": List[pd.DataFrame],
        "result_plots": List[PIL.Image],
    }
    ```
    где,
    - "semaphore" -- светофор выставленный за тест
    - "result_dict" -- python object, который валидатор посчитает полезным для дальнейшего использования
    - "result_dataframes" -- список таблиц, которые будут отражены в агрегированном excel файле
    - "result_plots" -- список картинок, которые будут отражены в агрегированном excel файле.
- ```sampler``` это единственный источник данных
- ```precomputed``` это Dict, который служит для обмена предпосчитанными значениями между тестами (зачем 5 раз считать метрику на oos, если можно 1 раз посчитать и записать это значение в dict)


### Цель:

Провести тест


In [11]:
# напишем какой-то тупой тест
# По длине train выставляет светофор

def custom_test(model, scorer, sampler, threshold = 1, precomputed=None, **kwargs):
    semaphore = 'green'
    if len(sampler.train['X']) < threshold:
        semaphore = 'red'
    
    df_table = pd.DataFrame({'len': [len(sampler.train['X'])],
                             'semaphore': semaphore})
    
    return {
        "semaphore": semaphore,
        "result_dict": {'train_shape':sampler.train['X'].values.shape},
        "result_dataframes": [df_table],
        "result_plots": [],
    }

custom_test(model, scorer, sampler, threshold=5, r_u_angry='True')

{'semaphore': 'green',
 'result_dict': {'train_shape': (700, 3)},
 'result_dataframes': [   len semaphore
  0  700     green],
 'result_plots': []}

## Pipeline (идея, цель, примеры использования)

### Идея:

Отражение понятия методики в нашей бибилиотеке. То есть ```Pipeline``` это список тестов с указанными параметрами и дополнительными метаданными для формирования отчета (*цель теста, интерпретация, способ агрегации блоков*).

### Цель:

Капсулировать всю информацию из методички


## Реализация:

```Pipeline``` - это excel-файл с двумя страницами
- первая страница -- информация о тестах
- вторая страница -- информация о блоках.

In [12]:
pd.read_excel("../src/sbe_vallib/table/pipelines/Config_31.xlsx", sheet_name="tests_config")

/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,import_path,test_key,block_key,informative,params,Название,Цель,Интерпретация,Границы красный,Границы желтый,Границы зеленый
0,sbe_vallib.table.data_quality.test_psi_factor....,test_factor_psi,data_quality,0,"{""merge_upto_quantile"": 0.05,\n ""rounding_prec...",Тест 1.2 Анализ однородности выборок out-of-sa...,Убедиться в однородности выборки (в разрезе фа...,Высокое значение PSI свидетельствует о различи...,Не выставляется,"PSI ≥ 0,2","PSI < 0,2"
1,sbe_vallib.table.model_quality.test_ci.test_ci,test_ci,model_quality,1,"{'metric_name': 'gini',\n 'n_iter': 200,\n 'ge...",Тест 2.2 Доверительный интервал ключевой метри...,Наглядно изобразить уровень статистической пог...,Данный тест позволяет оценить фактический разб...,Информативный,Информативный,Информативный
2,sbe_vallib.table.model_quality.test_key_metric...,test_key_metric,model_quality,0,"{'metric_name': 'gini', 'thresholds': (0.2, 0...",Тест 2.1 Анализ ключевой метрики качества (Gin...,Тест проводится для определения способности мо...,"Чем выше значение коэффициента Gini, тем лучше...","gini < 0,2",0.2 ≤ gini ≤ 0.4,gini > 0.4
3,sbe_vallib.table.stability.test_key_metric_sta...,test_key_metric_stability,stability,0,"{'metric_name': 'gini', 'abs_thresholds': (0....",Тест 5.1 Анализ стабильности модели на выборка...,Оценить стабильность модели по ключевой метрик...,"Чем слабее падает коэффициент Gini, тем стабил...",Абсолютное снижение более 25 п. п. и относите...,Абсолютное снижение более 15 п. п. И относител...,Абсолютное снижение менее 15 п. п. ИЛИ относит...


In [13]:
pd.read_excel("../src/sbe_vallib/table/pipelines/Config_31.xlsx", sheet_name="agg_config")

,block_key,print_name,func
0,data_quality,Качество данных,sbe_vallib.utils.report_helper.worst_semaphore
1,model_quality,Качество модели,sbe_vallib.utils.report_helper.worst_semaphore
2,calibration,Калибровка,sbe_vallib.utils.report_helper.worst_semaphore
3,specification,Спецификация,sbe_vallib.utils.report_helper.worst_semaphore
4,stability,Стабильность,sbe_vallib.utils.report_helper.worst_semaphore


Excel таблицы парсят вот в такой json, в Validation можно передавать excel и dict. На excel приятно смотреть, а dict приятней редачить

In [14]:
from sbe_vallib.parser import parse_pipeline

parse_pipeline("../src/sbe_vallib/table/pipelines/Config_31.xlsx")

/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


({'test_factor_psi': {'Границы зеленый': 'PSI < 0,2',
   'block_key': 'data_quality',
   'informative': 0,
   'Цель': 'Убедиться в однородности выборки (в разрезе факторов модели) для валидации (out-of-sample, out-of-time) относительно выборки для обучения по каждому классу',
   'Границы желтый': 'PSI ≥ 0,2',
   'import_path': 'sbe_vallib.table.data_quality.test_psi_factor.test_factor_psi',
   'Границы красный': 'Не выставляется',
   'Название': 'Тест 1.2 Анализ однородности выборок out-of-sample, out-of-time относительно train выборки (PSI) в разрезе факторов модели',
   'Интерпретация': 'Высокое значение PSI свидетельствует о различии в распределениях фактора на тестовой выборке по сравнению с выборкой для разработки, что может в числе прочих факторов свидетельствовать о потенциальной нестабильности модели. При высоких значениях PSI эта зона риска обязательно отражается в отчете о валидации',
   'params': {'merge_upto_quantile': 0.05,
    'rounding_precision_bins': 5,
    'discr_uniq

## Validation (идея, цель, примеры использования)

### Идея:

Запуск всех тестов из списка и аггрегация их результатов.

### Цель:

Удобный запуск тестов


<img src="./images/uml_vallib.jpg" alt="fishy" class="bg-primary mb-1" width="500px">

Можно добавлять в кастомные тесты в провдении валидации без внесения их в excel,
просто напишем для теста dict на примере того который получается после парсинга excel

```
{'test_key': {'block': 'test_block', 'callable': func, 'params': {'a': 4}}, 'informative': 0}
```

In [15]:
# способ прокидывания custim_test'ов в validation

from sbe_vallib.validation import Validation


custom_tests = {
    "custom_1": {"block": "data_quality", "callable": custom_test, "params": {"threshold": 5000}}
}


validor = Validation(model,
                     sampler,
                     scorer,
                     custom_tests=custom_tests,
                     pipeline="../src/sbe_vallib/table/pipelines/Config_31.xlsx")
res = validor.validate()
print('done')

/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Test: test_factor_psi started
Test: test_ci started
Test: test_key_metric started
Test: test_key_metric_stability started
	Test: test_key_metric_stability raised an exception:
Traceback (most recent call last):
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/validation.py", line 131, in validate
    tests_result[test_name] = test_function(
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/stability/test_key_metric_stability.py", line 57, in test_key_metric_stability
    metrics_oot = get_source_metrics(
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/model_quality/test_key_metric.py", line 11, in get_source_metrics
    if 'y_pred' not in data:
TypeError: argument of type 'NoneType' is not iterable

Test: custom_1 started
done


In [16]:
list(res.keys())

['test_factor_psi',
 'test_ci',
 'test_key_metric',
 'test_key_metric_stability',
 'custom_1']

In [17]:
res['test_factor_psi']['result_dataframes'][0]

,feature,psi,feat_type,bins,hist_train,hist_oos,semaphore
0,0,0.046485,Continuous,-inf; -2.48; -2.02; -1.56; -1.10; -0.64; -0.18...,0.00; 0.03; 0.08; 0.29; 0.53; 0.12; 0.15; 0.21...,0.00; 0.03; 0.12; 0.28; 0.51; 0.12; 0.14; 0.21...,green
1,1,0.05983,Continuous,-inf; -2.75; -2.37; -2.00; -1.63; -1.26; -0.89...,0.00; 0.00; 0.04; 0.06; 0.17; 0.25; 0.33; 0.37...,0.00; 0.02; 0.07; 0.07; 0.13; 0.29; 0.23; 0.47...,green
2,2,0.067574,Continuous,-inf; -3.13; -2.60; -2.07; -1.54; -1.02; -0.49...,0.00; 0.05; 0.10; 0.12; 0.22; 0.22; 0.23; 0.24...,0.00; 0.02; 0.08; 0.10; 0.21; 0.19; 0.18; 0.32...,green


In [18]:
print(res['test_key_metric_stability']['result_dataframes'][0]['error'][0])

Traceback (most recent call last):
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/validation.py", line 131, in validate
    tests_result[test_name] = test_function(
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/stability/test_key_metric_stability.py", line 57, in test_key_metric_stability
    metrics_oot = get_source_metrics(
  File "/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/model_quality/test_key_metric.py", line 11, in get_source_metrics
    if 'y_pred' not in data:
TypeError: argument of type 'NoneType' is not iterable

